# Get IWT country occurances, per trade route segment for the most mentioned species in the literature review database
### Code reads in a specifically formatted Excel file, and outputs both a count and list of IWT Country Occurances, per Trade Route Segment, per species.
### Output data is used for the top species mentioned across the literature chart.

In [7]:
import pandas as pd

# Read data
file_path = "Data/top_species_count.xlsx"
df = pd.read_excel(file_path, "Sheet1")

# Group data by species
df = df.map(lambda x: x.lower() if type(x) == str else x)
grouped = df.groupby("species")

# Function to filter missing (NaN, or "Not Specified)
def filter_countries(countries):
    return [country for country in countries if country != "not specified" and pd.notna(country)]

# Create empty results list
results = []

# Iterate through species
for species, group in grouped:
    
    # Get unique countries from each trade route stage
    supply_countries = filter_countries(group["Supply country"].unique().tolist())
    transit_countries = filter_countries(pd.concat([group["transit country 1"], group["transit country 2"], group["transit country 3"]]).unique().tolist())
    demand_countries = filter_countries(group["demand/consumer country"].unique().tolist())
    
    # Combine all countries
    all_countries = list(set(supply_countries + transit_countries + demand_countries))
    
    # Display countries at each per species (one species per row)
    row = {
        "Species": species.capitalize(),
        "Supply Countries Count": len(supply_countries),
        "Supply Countries": ", ".join(map(str, supply_countries)),
        "Transit Countries Count": len(transit_countries),
        "Transit Countries": ", ".join(map(str, transit_countries)),
        "Demand Countries Count": len(demand_countries),
        "Demand Countries": ", ".join(map(str, demand_countries)),
        "All Countries Count": len(all_countries),
        "All Countries": ", ".join(map(str, all_countries))
    }
    
    # Append the species row to the results
    results.append(row)

# Export the result
result_df = pd.DataFrame(results)
output_file_path = "Output/species_country_summary.xlsx"
result_df.to_excel(output_file_path, index=False)